In [4]:
# import required libraires 
from uk_covid19 import Cov19API
from IPython.display import clear_output
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wdg
import datetime 


# Retrieves json data from API, splits data in to two json files one with the covid data one with download data 
# Adds a key value pair to the download data with the time downloaded from the API
def getJson():
    apiData = api.get_json()
    covidData = wrangleData(apiData)
    covidData.to_json("CovidData.json")
    timeNow = str(pd.to_datetime("now"))
    apiData['lastDownload']= timeNow
    apiData.pop('data')
    with open("downloadData.json", "wt") as OUTF:
        json.dump(apiData, OUTF)
    return covidData

# Using Pandas methods, the data is inputted in to a dataframe, the date is column is change to a dateTime object 
# And is set as the index for each row. the index is then sorted in to ascending order and copies are deleted 
# All Null values are set to 0
def wrangleData(dataInput):
    data = dataInput['data']
    df = pd.DataFrame(data)
    # df.drop(df.index[0:500],inplace=True) # used to delete data for testing refresh_graph func 
    df.set_index('date', inplace = True)
    df.index = pd.to_datetime(df.index)
    df.index.duplicated(keep='first')
    df.sort_index(ascending=True, inplace=True)
    df.fillna(0, inplace = True)
    return df
   
# Called on button click, assigns the current df to the df from new API data, also calls a graph update
def access_api(button):
    global coviddf
    coviddf= getJson() # Reassign df object to new API data
    refresh_graph()
    apibutton.icon="check"
    apibutton.disabled=True

# Creates a graph using arguments form the widgets, determines what to display and at what time scale
def Covid_graph(graphColumns, graphIndex):
    coviddf.index[0]
    if graphIndex == 'All Time':
        timeScale = (coviddf.index[0],coviddf.index[-7])
    else:  
        timeScale = (coviddf.index[-69],coviddf.index[-7])
    ncols=len(graphColumns)
    if ncols>0:
        coviddf.plot( y=list(graphColumns))
        plt.xlim(timeScale)
        plt.show()
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# changes the values of the selectDate widget in order to force the graph to load the new API df
def refresh_graph():
    current=selectDate.value
    if current==selectDate.options[0]:
        other=selectDate.options[1]
    else:
        other=selectDate.options[0]
    selectDate.value=other 
    selectDate.value=current

# Assign a button from the ipywidgets library to the apibutton variable
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='',
    tooltip='Click to download current Public Health England data',
    icon='download' 
)

# Assign a SelectMultiple from the ipywidgets library to the selectData variable
selectData = wdg.SelectMultiple(
    options=['cases', 'critical', 'deaths'], # options available in widget 
    value=['cases', 'critical', 'deaths'], # defult value of widget
    rows=3,
    description='Data', # displayed name of widget
    disabled=False 
) 

# Assign a RadioButtons from the ipywidgets library to the selectDate variable
selectDate=wdg.RadioButtons(
    options=['Last 2 Months', 'All Time'],
    value='All Time',
    description='Scale:',
    disabled=False
)

# Covid Dashboard 

Graph showing official UK government coronavirus data for the United Kingdom


In [2]:
# The filter and structure define the metrics and structure in which the data will be receive from the API
filters = [
    'areaType=overview' 
]

structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "critical": "covidOccupiedMVBeds",
    "deaths": "newDeaths28DaysByDeathDate"
}

api = Cov19API(filters=filters, structure=structure)
coviddf = pd.read_json("CovidData.json") # Read the saved json file into a df object  

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

# display the API button and assign call back function 
display(apibutton)
apibutton.on_click(access_api)



Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…


The graph below compares the recorded covid cases, the number of covid patients in mechanical ventilation beds and the number of deaths within 28 days of positive test. 

These results can be seen all together or individually. The graphs can also be filtered to show either the whole covid period or the just the last 2 months (minus seven days, as results are released weekly)


In [3]:
controls=wdg.HBox([selectData, selectDate]) # Create a horizontal container for the widgets 
output=wdg.interactive_output(Covid_graph, {'graphColumns': selectData, 'graphIndex': selectDate}) # Create an interactive container for the widgets and call the Covid_graph func
display(controls, output) # Display graph and object containers 

Output()

*Author Graham Edwards - ec21955@qmul.ac.uk*